![Add a relevant banner image here](path_to_image)

# Project Title

## Overview

Short project description. Your bottom line up front (BLUF) insights.

## Business Understanding

Text here

## Data Understanding

Text here

In [3]:
# Load relevant imports here
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import gc

In [4]:
df_all_data = pd.read_csv('Data/US_Accidents_March23.csv')
print(df_all_data.describe())
print(df_all_data.head())
print(df_all_data.columns)

MemoryError: Unable to allocate 1.15 GiB for an array with shape (20, 7728394) and data type object

In [ ]:
df_severity_by_state = pd.crosstab(df_all_data['Severity'], df_all_data['State'])
df_severity_by_state.head()

In [ ]:
# chart of accident severity by state
states = df_severity_by_state.columns
sev1 = df_severity_by_state.iloc[0]
sev2 = df_severity_by_state.iloc[1]
sev3 = df_severity_by_state.iloc[2]
sev4 = ct_sev4_by_state = df_severity_by_state.iloc[3]

plt.figure(figsize=(12, 7))
plt.bar(states, sev1)
plt.bar(states, sev2, bottom=sev1)
plt.bar(states, sev3, bottom=sev1+sev2)
plt.bar(states, sev4, bottom=sev1+sev2+sev3)
plt.xlabel("State")
plt.xticks(rotation=60)
plt.ylabel("Number of Accidents")
plt.legend(["Severity 1", "Severity 2", "Severity 3", "Severity 4"])
plt.title("Accident Severity by State (2016 - 2023)")
plt.show()

In [ ]:
# chart of accident severity by state without sev2 (sev 2 >> than the others so obscures sev 1, 3, 4 above)
states = df_severity_by_state.columns
sev1 = df_severity_by_state.iloc[0]
sev3 = df_severity_by_state.iloc[2]
sev4 = ct_sev4_by_state = df_severity_by_state.iloc[3]

plt.figure(figsize=(12, 7))
plt.bar(states, sev1)
plt.bar(states, sev3)
plt.bar(states, sev4)
plt.xlabel("State")
plt.xticks(rotation=60)
plt.ylabel("Number of Accidents")
plt.legend(["Severity 1", "Severity 3", "Severity 4"])
plt.title("Accident Severity by State, Excluding Severity 2 (2016 - 2023)")
plt.show()

In [ ]:
df_sev_by_crossing = pd.crosstab(df_all_data['Severity'], df_all_data['Crossing'])
df_sev_by_crossing = df_sev_by_crossing.rename(columns={False: "No", True: "Yes"})
df_sev_by_crossing.head()

In [ ]:
# charts of accident severity by bump, traffic calming, roundabout
# chart of accident severity by state
crossing = df_sev_by_crossing.columns
cr1 = df_sev_by_crossing.iloc[0]
cr2 = df_sev_by_crossing.iloc[1]
cr3 = df_sev_by_crossing.iloc[2]
cr4 = df_sev_by_crossing.iloc[3]

plt.figure(figsize=(10, 7))
plt.bar(crossing, cr1)
plt.bar(crossing, cr2)
plt.bar(crossing, cr3)
plt.bar(crossing, cr4)
plt.xticks(rotation=60)
plt.xlabel("Nearby Crossing")
plt.ylabel("Number of Accidents")
plt.legend(["Severity 1", "Severity 2", "Severity 3", "Severity 4"])
plt.title("Accident Severity by Proximity to Crossing, (2016 - 2023)")
plt.show()

In [ ]:
print(f"Count of Wind Direction Entries: {df_all_data['Wind_Direction'].nunique()}")


## Data Preparation

### Data Selection

Based on my exploration of the data, I'm dropping the following fields from the dataset for the following reasons:

- Source: contains information that has no relationship to causes and effects of accidents
- Timezone: duplicates zip/state with less precision
- Country: all data is from the United States so this field is redundant
- Airport_Code: doesn't provide germane information-the exact location where weather conditions are reported is not a variable that can be adjusted
- Weather_Timestamp: not related to the conditions of the accidents in any way
- Wind_Direction: too many unique values; values are also not related to travel directions so it's unlikely they'll  produce clear/actionable conclusions


In [6]:
df_refined = df_all_data.drop(['Source', 'Timezone', 'Country', 'Airport_Code', 'Weather_Timestamp', 'Wind_Direction'], axis=1)

print(df_refined.describe())
print(df_refined.info())
print(df_refined.columns)

           Severity     Start_Lat     Start_Lng       End_Lat       End_Lng  \
count  7.728394e+06  7.728394e+06  7.728394e+06  4.325632e+06  4.325632e+06   
mean   2.212384e+00  3.620119e+01 -9.470255e+01  3.626183e+01 -9.572557e+01   
std    4.875313e-01  5.076079e+00  1.739176e+01  5.272905e+00  1.810793e+01   
min    1.000000e+00  2.455480e+01 -1.246238e+02  2.456601e+01 -1.245457e+02   
25%    2.000000e+00  3.339963e+01 -1.172194e+02  3.346207e+01 -1.177543e+02   
50%    2.000000e+00  3.582397e+01 -8.776662e+01  3.618349e+01 -8.802789e+01   
75%    2.000000e+00  4.008496e+01 -8.035368e+01  4.017892e+01 -8.024709e+01   
max    4.000000e+00  4.900220e+01 -6.711317e+01  4.907500e+01 -6.710924e+01   

       Distance(mi)  Temperature(F)  Wind_Chill(F)   Humidity(%)  \
count  7.728394e+06    7.564541e+06   5.729375e+06  7.554250e+06   
mean   5.618423e-01    6.166329e+01   5.825105e+01  6.483104e+01   
std    1.776811e+00    1.901365e+01   2.238983e+01  2.282097e+01   
min    0.000000e

### Data Cleaning

##### Missing Values

In [ ]:
df_refined['Start_Time'] = pd.to_datetime(df_refined['Start_Time'], yearfirst=True, format='mixed')
df_refined['End_Time'] = pd.to_datetime(df_refined['End_Time'], yearfirst=True, format='mixed')
df_refined['Acc_date'] = df_refined['Start_Time'].dt.date
df_refined.tail(5)

In [23]:
# replacing NaN values in the temperature column

def fill_missing_temperatures(df, date_col='Acc_date', zip_col='Zipcode', state_col='State', temp_col='Temperature(F)'):
    """
    Replaces NaN Temperature values with the mean temperature of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean temperature for each date-zip combination
    df_temp_fill = df_refined.copy()
    
    temp_means_zip = df_temp_fill.groupby([date_col, zip_col])[temp_col].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_temp_fill[temp_col] = df_temp_fill[temp_col].fillna(temp_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_temp_fill[temp_col].isna().any():
        remaining = df_temp_fill[temp_col].isna().sum()
        print(f"Info: {remaining} temperatures still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        temp_means_state = df_temp_fill.groupby([date_col, state_col])[temp_col].transform('mean')
        df_temp_fill[temp_col] = df_temp_fill[temp_col].fillna(temp_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_temp_fill[temp_col].isna().any():
        remaining = df_temp_fill[temp_col].isna().sum()
        print(f"Warning: {remaining} temperatures still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_temp_fill[temp_col] = df_temp_fill[temp_col].fillna(df_temp_fill[temp_col].mean())
    
    return df_temp_fill

df_temp_fill = fill_missing_temperatures(df_refined)

Info: 158386 temperatures still missing after date-zip fill.
Filling remaining with date-state mean.
Filling remaining with overall mean as last resort.


In [ ]:
# verifying NaNs have been replaced in the working df
df_temp_fill['Temperature(F)'].isna().sum()

0

In [ ]:
# writing NaN-free column back to the original dataframe
df_refined['Temperature(F)'] = df_temp_fill['Temperature(F)']
df_refined['Temperature(F)'].isna().sum()

0

In [ ]:
# deleting working df to free up memory
del df_temp_fill
gc.collect()

In [26]:
# replacing NaN values in the wind chill column

def fill_missing_windchill(df, date_col='Acc_date', zip_col='Zipcode', state_col='State', windchill_col='Wind_Chill(F)'):
    """
    Replaces NaN wind chill values with the mean wind chill of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean temperature for each date-zip combination
    df_windchill_fill = df_refined.copy()
    
    windchill_means_zip = df_windchill_fill.groupby([date_col, zip_col])[windchill_col].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_windchill_fill[windchill_col] = df_windchill_fill[windchill_col].fillna(windchill_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_windchill_fill[windchill_col].isna().any():
        remaining = df_windchill_fill[windchill_col].isna().sum()
        print(f"Info: {remaining} wind chills still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        windchill_means_state = df_windchill_fill.groupby([date_col, state_col])[windchill_col].transform('mean')
        df_windchill_fill[windchill_col] = df_windchill_fill[windchill_col].fillna(windchill_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_windchill_fill[windchill_col].isna().any():
        remaining = df_windchill_fill[windchill_col].isna().sum()
        print(f"Warning: {remaining} wind chills still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_windchill_fill[windchill_col] = df_windchill_fill[windchill_col].fillna(df_windchill_fill[windchill_col].mean())
    
    return df_windchill_fill

df_windchill_fill = fill_missing_windchill(df_refined)

Info: 1949795 wind chills still missing after date-zip fill.
Filling remaining with date-state mean.
Filling remaining with overall mean as last resort.


In [ ]:
# verifying NaNs have been replaced in the working df
df_windchill_fill['Wind_Chill(F)'].isna().sum()

0

In [ ]:
# writing NaN-free column back to the original dataframe
df_refined['Wind_Chill(F)'] = df_windchill_fill['Wind_Chill(F)']
df_refined['Wind_Chill(F)'].isna().sum()

0

In [ ]:
# deleting working df to free up memory
del df_windchill_fill
gc.collect()

In [29]:
# replacing NaN values in the humidity column

def fill_missing_hum(df, date_col='Acc_date', zip_col='Zipcode', state_col='State', hum_col='Humidity(%)'):
    """
    Replaces NaN humidity values with the mean humidity of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean humidity for each date-zip combination
    df_hum_fill = df_refined.copy()
    
    hum_means_zip = df_hum_fill.groupby([date_col, zip_col])[hum_col].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_hum_fill[hum_col] = df_hum_fill[hum_col].fillna(hum_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_hum_fill[hum_col].isna().any():
        remaining = df_hum_fill[hum_col].isna().sum()
        print(f"Info: {remaining} humidity entries still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        hum_means_state = df_hum_fill.groupby([date_col, state_col])[hum_col].transform('mean')
        df_hum_fill[hum_col] = df_hum_fill[hum_col].fillna(hum_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_hum_fill[hum_col].isna().any():
        remaining = df_hum_fill[hum_col].isna().sum()
        print(f"Warning: {remaining} humidity entries still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_hum_fill[hum_col] = df_hum_fill[hum_col].fillna(df_hum_fill[hum_col].mean())
    
    return df_hum_fill

df_hum_fill = fill_missing_hum(df_refined)

MemoryError: Unable to allocate 708. MiB for an array with shape (12, 7728394) and data type float64

In [ ]:
# verifying NaNs have been replaced in the working df
df_hum_fill['Humidity(%)'].isna().sum()

In [ ]:
# writing NaN-free column back to the original dataframe
df_refined['Humidity(%)'] = df_hum_fill['Humidity(%)']
df_refined['Humidity(%)'].isna().sum()

In [ ]:
# deleting working df to free up memory
del df_hum_fill
gc.collect()

In [ ]:
# replacing NaN values in the pressure column

def fill_missing_press(df, date_col='Acc_date', zip_col='Zipcode', state_col='State', press_col='Pressure(in)'):
    """
    Replaces NaN pressure values with the mean pressure of entries
    with the same date and zip code. If no match exists with date and county,
    uses date and state.
    """
    # Step 1: mean pressure for each date-zip combination
    df_press_fill = df_refined.copy()
    
    press_means_zip = df_press_fill.groupby([date_col, zip_col])[press_col].transform('mean')
    
    # fill NaN values with the date-zip group mean
    df_press_fill[press_col] = df_press_fill[press_col].fillna(press_means_zip)
    
    # Step 2: date-state combination for remaining NaNs
    if df_press_fill[press_col].isna().any():
        remaining = df_press_fill[press_col].isna().sum()
        print(f"Info: {remaining} pressure entries still missing after date-zip fill.")
        print("Filling remaining with date-state mean.")
        
        press_means_state = df_press_fill.groupby([date_col, state_col])[press_col].transform('mean')
        df_press_fill[press_col] = df_hum_fill[press_col].fillna(press_means_state)
    
    # Step 3: if any NaNs still remain, fill with overall mean as last resort
    if df_press_fill[press_col].isna().any():
        remaining = df_press_fill[press_col].isna().sum()
        print(f"Warning: {remaining} pressure entries still missing after date-state fill.")
        print("Filling remaining with overall mean as last resort.")
        df_press_fill[press_col] = df_press_fill[[press]_col].fillna(df_press_fill[press_col].mean())
    
    return df_hum_fill

df_hum_fill = fill_missing_hum(df_refined)

I've managed missing values in the code blocks above. Here's a brief explanation of my approch to each column:

- Temperature, Wind_Chill, Humidity, Pressure, Visibility: imputed based on the mean temp of other accident entries sharing the same day and zip code (or state if there are none in the zip code)
- Precipitation, Wind_Speed: assumed NaN indicates no precipitation/wind and replaced NaN with zero
- Weather_Condition: consolidated entries from 144 to 
- Sunrise_Sunset: imputed based on
- Civil_Twilight: imputed based on
- Nautical_Twilight: imputed based on
- Astronomical_Twilight: imputed based on

In [ ]:
#print(f"Count of Unique Weather Condition Entries: {df_all_data['Weather_Condition'].nunique()}")
df_refined['Weather_Condition'].value_counts().sort_values(ascending=True)



## Analysis

Text here

## Evaluation

### Business Insight/Recommendation 1

### Business Insight/Recommendation 2

### Business Insight/Recommendation 3

### Tableau Dashboard link

## Conclusion and Next Steps
Text here